# Importing Modules

In [10]:
import pandas as pd
import numpy as np 
import xlsxwriter 
import requests
import math

# Obtaining Stock Information

In [11]:
stocks = pd.read_csv('sp_500_stocks.csv')
#type(stocks)
stocks.head()

,Ticker
0,AAP
1,AAPL
2,ABBV
3,ABC
4,ABMD


# Acquiring an API Token

In [12]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

# Making our first API Call

In [13]:
symbol = 'AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

print(data)

{'avgTotalVolume': 58889141, 'calculationPrice': 'tops', 'change': -0.4, 'changePercent': -0.0035, 'close': 0, 'closeSource': 'ocaififl', 'closeTime': None, 'companyName': 'Amazon.com Inc.', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 117.3, 'iexAskSize': 110, 'iexBidPrice': 117.78, 'iexBidSize': 208, 'iexClose': 119.6, 'iexCloseTime': 1744405187019, 'iexLastUpdated': 1724859672021, 'iexMarketPercent': 0.02540823661933146, 'iexOpen': 119.37, 'iexOpenTime': 1747299264827, 'iexRealtimePrice': 119.1, 'iexRealtimeSize': 102, 'iexVolume': 923114, 'lastTradeTime': 1747452095702, 'latestPrice': 118.3, 'latestSource': 'IEX real time price', 'latestTime': '3:09:31 PM', 'latestUpdate': 1702639395658, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 1168618083970, 'oddLotDelay

# Parsing our API Call

In [14]:
price = data['latestPrice']
company_name = data['companyName']

company_name, price

('Amazon.com Inc.', 118.3)

# Adding our stocks data to a pandas dataframe

In [15]:
my_columns = ['Ticker','Company Name', 'Stock Price', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Company Name,Stock Price,Number of Shares to Buy


In [16]:
final_dataframe.append(
        pd.Series(
            [
                symbol,
                
                company_name,
                
                price,
                'N/A'
            ],
            index = my_columns
            ), 
            ignore_index=True )


,Ticker,Company Name,Stock Price,Number of Shares to Buy
0,AMZN,Amazon.com Inc.,118.3,N/A


# Looping Through The Tickers in Our List of Stocks

In [17]:
# In the future I'll add batch API Request for speed improvement.
'''
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:    
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        'N/A'
    ],
    index = my_columns),
    ignore_index = True)
    
    final_dataframe.head()
    '''

"\nfinal_dataframe = pd.DataFrame(columns = my_columns)\nfor stock in stocks['Ticker']:    \n    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'\n    data = requests.get(api_url).json()\n    final_dataframe = final_dataframe.append(\n    pd.Series(\n    [\n        stock,\n        data['latestPrice'],\n        'N/A'\n    ],\n    index = my_columns),\n    ignore_index = True)\n    \n    final_dataframe.head()\n    "

# Using Batch API Calls to improve performance

In [18]:
#1.Split the list of tickers in sublists.

#2. Using chuncks function
#2.1. Source: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

#3. See batch requests in IEX API documentation

def chunks(lst,n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [19]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    
    symbol_strings.append(','.join(symbol_groups[i]))
    #print (symbol_strings[i])
    
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]: 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['companyName'],
            data[symbol]['quote']['latestPrice'],            
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index = True
        )

# Calculating the number of shares to buy to reach portifolio_size

In [22]:
portifolio_size = 100000
val = float(portifolio_size)

position_size = val/len(final_dataframe.index)
print(position_size)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])

print (final_dataframe.tail())

1000.0
   Ticker             Company Name  Stock Price Number of Shares to Buy
95    CMA           Comerica, Inc.       74.050                      13
96  CMCSA   Comcast Corp - Class A       30.185                      33
97    CME  CME Group Inc - Class A      180.360                       5
98    CMG   Chipotle Mexican Grill     1576.820                       0
99    CMI             Cummins Inc.      206.543                       4
